In [1]:
import os
from time import sleep

import pandas as pd
from sec_edgar_downloader import Downloader
from tqdm import tqdm

In [2]:
# places to put files - best practice chapter 2!

os.makedirs("input", exist_ok=True)
os.makedirs("10k_files", exist_ok=True)

In [3]:
# download sample (somewhat simplistic option!)

sample_csv = "input/sp500_firms.csv"
if not os.path.exists(sample_csv):
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    sample = pd.read_html(url)[0]
    sample.to_csv(sample_csv)

## Issues to consider with our spider below (the loop that downloads all the 10-Ks)

- [x] loop over cik or ticker?
    - here, tickers, but BEWARE! 
- [x] too fast! (maybe?)
    - slow the code down!
- [x]  everytime I run dl.get(), it redownloads 
    - don't! if i already downloaded the APPL don't repeat
- [x] progress unclear - use a progress tracker 
    - manual prints
    - or tqdm()
- [ ] huge memory requirments! on my computer it would be 15GB!
    - option a: lets only keep html files, and delete the txt files as we go (reduces to ~2.5GB)
    - option b: put all the html files in a zipfolder (**much** less space)
    - option c: a+b
    - option d: extract the text from the file AS you download, which reduces how much you save on your computer (instead of the whole raw file). You can even put those in a zip file if you want. The main issue is that at the beginning of a large scraping task, it's hard to know if your code to extract the text will work across all the files you want. Since "hard drives are cheap", I usually just grab all the data at the front, and deal with it after. 
- [x] huge memory requirments! on github 
    - gitignore


## This loop is a spider!

The main thing left to figure out is how to delete the text files as you go. 
- Then do the loop for 10 firms.
- Then 20. (Notice how the first 10 aren't downloaded again!)
- Look at the file size growing. Multiply the size of the 10k_files folder by 500/20 to guesstimate the full size. I'm guessing around 2.5GB.
- If you want to zip the files, one way to do that is to
    - use `shutil` to zip the whole directory after you finish the downloads
    - then delete the directory
    - you'll have to modify the measure_risk code to open 10-Ks from the zip file using `ZipFile`

In [3]:
dl = Downloader('10k_files')

In [4]:
firms = ["MSFT", "AMZN", "AAPL", "TSLA"]

for firm in tqdm(firms):

    # an overly simply option that might be too restrictive on bolder projects
    # just look to see if we have a folder for that firm's 10-K and assume
    # that means we have the 10-k for the right date (we could write a way to
    # verify that)
    firm_folder = "10k_files/sec-edgar-filings/" + firm
    if not os.path.exists(firm_folder):
        dl.get("10-K", firm, amount=1, after="2019-03-01", before="2020-03-21")

    # pause - be nice to server (probably unneeded, the sec_edgar_downloader
    # source code contains checks to limit speed)
    sleep(0.5)

    # if I have downloaded txt files inside firm_folder:
    # get list of .txt in the firms folder
    # print them! once you're printing them, you can just...
    # delete them

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.87s/it]
